# Load GEMD files in database
This code is for information purposes onlny as it requires permissions on our SciServer system that users of this repo will not in general posess. The database schema is defined by files in the database_design subfolder.

#### NB parts of this code can only be run by someone with direct credentials to the database

In [ ]:
import glob
import json
import random
import pandas
import pprint
import sys
import json
import numpy as np
import pymssql

In [ ]:
from gemd_database import MSSQLDatabase

In [ ]:
with open('/home/idies/workspace/persistent/.private/gemd.json','r') as f:
    AUTH = json.load(f)
GEMDDB=MSSQLDatabase(AUTH,'GEMD')

# add more explicit tables and views
For each of the main object types create table/view and extract somewhat more explicit metadata into columns. In particular a name and all the references/foreign keys to other tables n this set. Note that the tables are defined implicitly using the <tt>SELECT ... INTO...</tt> supported by MS SQL. For this reason one needs a database user with appopriate privileges.

<img src='https://citrineinformatics.github.io/gemd-docs/img/GEMD-Object-Graph.png'></img>


Use class in utility script. Again must be laoded with appropriate permissions not avilable to random users.

## material

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      context
  into material_template
  from gemdobject
 where gemd_type='material_template'
"""
r=GEMDDB.execute_update(sql)    

In [ ]:
sql="""
--drop table material_spec
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      json_value(context,'$.process.id') as process_spec_uid
,      context
  into material_spec
  from gemdobject
 where gemd_type='material_spec'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.spec.id') as spec_uid
,      json_value(context,'$.process.id') as process_run_uid
,      context
  into material_run
  from gemdobject
 where gemd_type='material_run'
"""
GEMDDB.execute_update(sql)

## measurement

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      context
  into measurement_template
  from gemdobject
 where gemd_type='measurement_template'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      context
  into measurement_spec
  from gemdobject
 where gemd_type='measurement_spec'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.source.performed_by') as performed_by
,      json_value(context,'$.source.performed_date') as performed_date
,      json_value(context,'$.spec.id') as spec_uid
,      json_value(context,'$.material.id') as material_uid
,      context
  into measurement_run
  from gemdobject
 where gemd_type='measurement_run'
"""
GEMDDB.execute_update(sql)

## process

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      context
  into process_template
  from gemdobject
 where gemd_type='process_template'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.source.performed_by') as p_performed_by
,      json_value(context,'$.source.performed_date') as p_performed_date
,      json_value(context,'$.spec.id') as spec_uid
,      context
  into process_run
  from gemdobject
 where gemd_type='process_run'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      context
  into process_spec
  from gemdobject
 where gemd_type='process_spec'
"""
GEMDDB.execute_update(sql)

## ingredient

In [ ]:
# into ingredient_spec
#   
sql="""
select distinct s.gemd_type as spec
,      s.model_id
, m.gemd_type as material, p.gemd_type as process
  from gemdobject i 
  join gemdobject s on s.uid=json_value(i.context,'$.spec.id')
  join gemdobject m on m.uid=json_value(i.context,'$.material.id')
  join gemdobject p on p.uid=json_value(i.context,'$.process.id')
 where i.gemd_type='ingredient_run'
"""
GEMDDB.execute_query(sql)
# GEMDDB.execute_update(sql)

In [ ]:
# 
#   
sql="""
select i.uid
,      model_id
,      json_value(i.context,'$.name') as name
,      json_value(i.context,'$.material.id') as material_spec_uid
,      json_value(i.context,'$.process.id') as process_spec_uid
,      i.context
  into ingredient_spec
  from gemdobject i 
where i.gemd_type='ingredient_spec'
"""
# GEMDDB.execute_query(sql)
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select i.uid
,      model_id
,      json_value(i.context,'$.name') as name
,      json_value(i.context,'$.spec.id') as spec_uid
,      json_value(i.context,'$.material.id') as material_run_uid
,      json_value(i.context,'$.process.id') as process_run_uid
,      i.context
  into ingredient_run
  from gemdobject i 
where i.gemd_type='ingredient_run'
"""
# GEMDDB.execute_query(sql)
GEMDDB.execute_update(sql)

## attribute_template
generic for ['condition_template','property_template','parameter_template']

In [ ]:
sql = """
select gc.uid, gc.model_id, gc.gemd_type
,      b.name, b.description
,      b.bounds_type, b.units
,      case when bounds_type = 'real_bounds' or bounds_type = 'integer_bounds' then
              JSON_QUERY((
              select lower_bound = JSON_VALUE(b.bounds,'$.lower_bound')
              ,      upper_bound = JSON_VALUE(b.bounds,'$.upper_bound')
                for json path
              ))
            when bounds_type = 'categorical_bounds' then
              JSON_QUERY(b.bounds,'$.categories')
            when bounds_type = 'composition_bounds' then
              JSON_QUERY(b.bounds,'$.components')
            else null end as bounds
,      b.tags
,      gc.context
into attribute_template
  from gemdobject gc
  cross apply OPENJSON(gc.context,'$') with (
    name nvarchar(128) '$.name'
  , description nvarchar(max) '$.description'
  , bounds_type nvarchar(32) 'lax$.bounds.type'
  , units nvarchar(32) 'lax$.bounds.default_units'
  , tags nvarchar(max) 'lax$.bounds.default_units' as json
  , bounds nvarchar(max) 'lax$.bounds' as json) b
 where gc.gemd_type in ('condition_template','property_template','parameter_template')
"""
GEMDDB.execute_update(sql)